## Meteo to nc

In [110]:
from Poseidon.utils import *
from Poseidon.meteo import *
from netCDF4 import Dataset
import datetime

### setup

In [111]:
#define in a dictionary the properties e.g
dic={'lon0':0., # lat/lon window
     'lon1':10.,
     'lat0':28.,
     'lat1':38.}

### extract meteo forcing

In [115]:
#set meteo forcing
m = ecmwf() # initialize

In [118]:
# specify the forecast times ft1,ft2
m.ft1 = 0
m.ft2 = 2 #that is 0,1 hours

In [119]:
# parse the ecmwf grib file.
m.parse(path='/Users/brey/Downloads/20130301.00.tropical_cyclone.grib',**dic)

100%|██████████| 6/6 [00:02<00:00,  2.33it/s]


meteo done


## output 

Below the output specifically for Schism but one can adapt appropriately

In [131]:
#from DataFile class create a file
d = DataFile('/Users/brey/Downloads/test.nc', write=True, create=True,format='NETCDF3_64BIT')

In [132]:
## add OPTIONAL attributes to the nc file
info = {'description' : '',
    'history' :'JRC Ispra European Commission',
    'source' : 'netCDF4 python module'}

In [133]:
d.info(**info)

In [134]:
tstamp = datetime.datetime(2013,3,1,0)

In [135]:
tu = datetime.datetime.strftime(tstamp,'%Y-%m-%d' )
bd = np.array([tstamp.year,tstamp.month,tstamp.day,tstamp.hour])

In [136]:
time = np.arange(m.ft1,m.ft2)/24. +tstamp.hour/24.

In [ ]:
tt=time.astype(np.float32)
pp=m.p.astype(np.float32)
uu=m.u.astype(np.float32)
vv=m.v.astype(np.float32)
lons=m.lons.astype(np.float32)
lats=m.lats.astype(np.float32)

In [137]:
spfh = np.empty(vv.shape)

In [138]:
#Write variables
d.write('time',tt, ['time'], **{'limited':False,'long_name': 'Time', 'standard_name': 'time', 'units': tu,'base_date': bd})


d.write('lat',lats,  **{'vname':['nx_grid','ny_grid'],'long_name': 'Latitude', 'standard_name': 'latitude', 'units': 'degrees_north'})
d.write('lon',lons, **{'vname':['nx_grid','ny_grid'], 'long_name': 'Longitude', 'standard_name': 'longitude', 'units': 'degrees_east'})
d.write('prmsl',pp,  **{'vname':['time','nx_grid','ny_grid'],'long_name': 'Pressure reduced to MSL', 'standard_name': 'air_pressure_at_sea_level', 'units': 'Pa'})
d.write('uwind',uu, **{'vname':['time','nx_grid','ny_grid'], 'long_name': 'Surface Eastward Air Velocity', 'standard_name': 'eastward_wind', 'units': 'm/s'})
d.write('vwind',vv, **{'vname':['time','nx_grid','ny_grid'], 'long_name': 'Surface Northward Air Velocity', 'standard_name': 'northward_wind', 'units': 'm/s'})
d.write('spfh',spfh, **{'vname':['time','nx_grid','ny_grid'], 'long_name': 'Surface Specific Humidity (2m AGL)', 'standard_name': 'specific_humidity', 'units': '1'})
d.write('stmp',spfh, **{'vname':['time','nx_grid','ny_grid'], 'long_name': 'Surface Temperature', 'standard_name': 'surface temperature', 'units': 'degrees'})

In [139]:
#close
d.close()

In [140]:
#check
q = Dataset('/Users/brey/Downloads/test.nc')

In [141]:
q.variables['time']

<type 'netCDF4._netCDF4.Variable'>
float32 time(time)
    units: 2013-03-01
    standard_name: time
    base_date: [2013    3    1    0]
    long_name: Time
unlimited dimensions: time
current shape = (2,)
filling off